In [3]:
import gym
from time import sleep, time
from IPython.display import clear_output

env = gym.make('Taxi-v3', render_mode='ansi')
env.reset()

def bfs(env, start):
    queue = [(start, [start])]
    while queue:
        state, path = queue.pop(0)
        for action in range(env.action_space.n):
            env.unwrapped.s = state
            new_state, reward, done, _, _ = env.step(action)

            if done:
                #yield path + [new_state]   # this would be to find all possible paths
                return path + [new_state]   # return the first one (the shortest one in case of BFS)
            elif new_state not in path:
                queue.append((new_state, path + [new_state]))

In [4]:
t0 = time()
shortest_path = bfs(env, start=328)
t = time()-t0

for state in shortest_path:
    env.reset()
    env.unwrapped.s = state
    clear_output()
    print(env.render())
    sleep(0.42)

print(f'Shortest path of len {len(shortest_path)} found in {t} seconds.')
print(f'>> {shortest_path}')

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


Shortest path of len 11 found in 0.054148197174072266 seconds.
>> [328, 228, 208, 308, 408, 416, 316, 216, 116, 16, 0]
